In [1]:
# Import the NetworkX package
import networkx as nx

import torch
print("PyTorch has version {}".format(torch.__version__))

PyTorch has version 1.10.2


In [2]:
!pwd

/Users/wangshihao/Research/SocialNetwork/HW1


In [3]:
# create BC dict
f = open('../data/hw1_data/Synthetic/5000/0_score.txt')
bc_dict = dict()
for line in f:
    line = line.replace("\n", "")
    a, b = line.split("\t")
    bc_dict[int(a)] = float(b)
f.close()
len(bc_dict.keys())

5000

In [4]:
f = open('../data/hw1_data/Synthetic/5000/0.txt')
text = []
for line in f:
    line = line.replace("\n", "")
    a, b = line.split("\t")
    text.append((int(a), int(b)))
f.close()

## 1. Graph

In [19]:
# Create an undirected graph G
G = nx.Graph(text)
print(G.is_directed())

False


In [20]:
# Get number of nodes
num_nodes = G.number_of_nodes()
print("G has {} nodes".format(num_nodes))

# Get number of edges
num_edges = G.number_of_edges()
print("G has {} edges".format(num_edges))

G has 5000 nodes
G has 19982 edges


In [21]:
node_id = 1

# Degree of node 1
print("Node {} has degree {}".format(node_id, G.degree[node_id]))

# Get neighbor of node 1
#for neighbor in G.neighbors(node_id):
#    print("Node {} has neighbor {}".format(node_id, neighbor))

Node 1 has degree 178


In [30]:
# set x feature
dergees = {node:[val, 1, 1] for (node, val) in G.degree()}
nx.set_node_attributes(G, dergees, "x")
nx.set_node_attributes(G, bc_dict, "y")
G.nodes[0]

{'x': [239, 1, 1], 'y': 0.09417453090592563}

In [458]:
def read_Data(num):
    # create BC dict
    f = open(f'../data/hw1_data/Synthetic/5000/{num}_score.txt')
    bc_dict = dict()
    for line in f:
        line = line.replace("\n", "")
        a, b = line.split("\t")
        bc_dict[int(a)] = float(b)
    f.close()
    len(bc_dict.keys())
    
    # read node pair
    f = open(f'../data/hw1_data/Synthetic/5000/{num}.txt')
    text = []
    for line in f:
        line = line.replace("\n", "")
        a, b = line.split("\t")
        text.append((int(a), int(b)))
    f.close()
    
    # Create an undirected graph G
    G = nx.Graph(text)
    
    dergees = {node:[val, 1, 1] for (node, val) in G.degree()}
    nx.set_node_attributes(G, dergees, "x")
    nx.set_node_attributes(G, bc_dict, "y")
    
    return G

## 2. Pytorch Geometric

In [37]:
from torch_geometric.utils.convert import from_networkx
# Convert the graph into PyTorch geometric
pyg_graph = from_networkx(G)
print(pyg_graph)
print(pyg_graph.x)
print(pyg_graph.y)
print(pyg_graph.edge_index)
print(f'Number of features: {pyg_graph.num_features}')

data = pyg_graph  # Get the first graph object.

print(data)
print('==============================================================')

# Gather some statistics about the graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {(2*data.num_edges) / data.num_nodes:.2f}')

Data(x=[5000, 3], edge_index=[2, 39964], y=[5000])
tensor([[239,   1,   1],
        [196,   1,   1],
        [220,   1,   1],
        ...,
        [  4,   1,   1],
        [  4,   1,   1],
        [  4,   1,   1]])
tensor([9.4175e-02, 7.6438e-02, 9.2790e-02,  ..., 2.8613e-05, 2.5731e-05,
        2.0743e-05])
tensor([[   0,    0,    0,  ..., 4999, 4999, 4999],
        [   1,    2,    3,  ..., 2523, 4508, 4803]])
Number of features: 3
Data(x=[5000, 3], edge_index=[2, 39964], y=[5000])
Number of nodes: 5000
Number of edges: 39964
Average node degree: 15.99


## 2.1. Model

In [51]:
edge_index = data.edge_index
print(edge_index.t())

tensor([[   0,    1],
        [   0,    2],
        [   0,    3],
        ...,
        [4999, 2523],
        [4999, 4508],
        [4999, 4803]])


In [459]:
import torch
from torch.nn import GRU
from torch_geometric.nn import GCNConv

L = 5
class Encorder(torch.nn.Module):
    def __init__(self):
        super(Encorder, self).__init__()
        torch.manual_seed(12345)
        self.input = GCNConv(pyg_graph.num_features, 128)
        self.conv = GCNConv(128, 128)
        self.gru = GRU(128, 128, bias=False)

    def forward(self, x, edge_index):
        # h1
        h1 = self.input(x, edge_index)
        h1 = h1.relu()
        hi_1 = torch.nn.functional.normalize(h1, p=2, dim=1)
        # h2 ~ L
        stack_h = hi_1.reshape(1, -1, 128)
        for epoch in range(L):
            for i in range(x.shape[0]):
                # Caculate Neighbor
                indices = torch.nonzero(edges[0]==i)
                neighbor_index = edges[1,torch.flatten(indices)]
                n_degree = pyg_graph.x[neighbor_index, 0]
                v_degree = pyg_graph.x[i, 0] #x[i, 0]
                dj = torch.sqrt(n_degree+1)
                dv = torch.sqrt(v_degree+1)
                normal_n = torch.div(hi_1[neighbor_index].reshape(128, -1), dv * dj)
                h_n = torch.sum(normal_n, 1)

                # GRU Cell
                hv = hi_1[i]
                hv, _ = self.gru(hv.reshape(1, 1, 128), h_n.reshape(1, 1, 128)) # (input, hidden)
                # concat GRU result
                if i == 0:
                    v_cat = hv
                else: 
                    v_cat = torch.cat([v_cat, hv], dim=0)

            # normalize new hi_1
            hi_1 = v_cat.reshape(-1, 128)
            hi_1 = torch.nn.functional.normalize(h1, p=2, dim=1)
            # store hidden result
            h_cat = torch.cat([stack_h, hi_1.reshape(1, -1, 128)], dim=0)
        # z is maximum from all hidden values
        z, _ = torch.max(h_cat, 0)
        
        return z

encorder = Encorder()
print(encorder)

Encorder(
  (input): GCNConv(3, 128)
  (conv): GCNConv(128, 128)
  (gru): GRU(128, 128, bias=False)
)


In [442]:
out= encorder(pyg_graph.x.float(), pyg_graph.edge_index)
print(out)

tensor([[0.0000, 0.0641, 0.0000,  ..., 0.0000, 0.1850, 0.0847],
        [0.0000, 0.0650, 0.0000,  ..., 0.0000, 0.1848, 0.0838],
        [0.0000, 0.0653, 0.0000,  ..., 0.0000, 0.1848, 0.0836],
        ...,
        [0.0000, 0.0557, 0.0000,  ..., 0.0000, 0.1854, 0.0920],
        [0.0000, 0.0545, 0.0000,  ..., 0.0000, 0.1853, 0.0929],
        [0.0000, 0.0553, 0.0000,  ..., 0.0000, 0.1854, 0.0923]],
       grad_fn=<MaxBackward0>)


In [444]:
from torch.nn import Linear
class Decorder(torch.nn.Module):
    def __init__(self):
        super(Decorder, self).__init__()
        torch.manual_seed(12345)
        self.linear = Linear(128, 32)
        self.output = Linear(32, 1)

    def forward(self, embedding):
        h = self.linear(embedding)
        h = h.relu()
        out = self.output(h)
        
        return out
    
decorder = Decorder()
print(decorder)

Decorder(
  (linear): Linear(in_features=128, out_features=32, bias=True)
  (output): Linear(in_features=32, out_features=1, bias=True)
)


In [446]:
y_hat= decorder(out)
print(y_hat)

tensor([[0.2054],
        [0.2053],
        [0.2053],
        ...,
        [0.2055],
        [0.2055],
        [0.2055]], grad_fn=<AddmmBackward0>)


In [518]:
from random import randint 

def sample_node(y_hat, edge_index, sample_num = 10000):
    loss = 0
    sigmoid = torch.nn.Sigmoid()
    for i in range(sample_num):
        idx = randint(0, edge_index.shape[1]-1)
        rand_edge = edge_index[:,idx]
        # loss
        yij = sigmoid(y_hat[rand_edge[0]] - y_hat[rand_edge[1]])
        bij = sigmoid(pyg_graph.y[rand_edge[0]] - pyg_graph.y[rand_edge[1]].reshape(1))
        if i == 0:
            yij_cat = yij
            bij_cat = bij
        else:
            yij_cat = torch.cat((yij_cat, yij))
            bij_cat = torch.cat((bij_cat, bij))
        #loss += -sigmoid(bij)*torch.log(sigmoid(yij)) - (1-sigmoid(bij))*torch.log(1-sigmoid(yij))
    
    return yij_cat, bij_cat
yij, bij = sample_node(y_hat, edges)
loss = criterion(yij.reshape(-1, 1), bij.reshape(-1, 1))
loss

tensor(-0., grad_fn=<DivBackward1>)

In [ ]:
# training
lr = 0.0001
MAX_ITERATION = 10000
encorder = Encorder()
decorder = Decorder()
criterion = torch.nn.CrossEntropyLoss()  # Define loss criterion.
encorder_optimizer = torch.optim.Adam(encorder.parameters(), lr=lr)
decorder_optimizer = torch.optim.Adam(decorder.parameters(), lr=lr)

for i in range(MAX_ITERATION):
    G = get_training_data()
    pyg_graph = from_networkx(G)
    edges = pyg_graph.edge_index

    # get each node's embedding z
    z= encorder(pyg_graph.x.float(), pyg_graph.edge_index)
    # compute BC ranking score
    y_hat= decorder(z)
    # compute loss
    yij, bij = sample_node(y_hat, edges)
    loss = criterion(yij.reshape(-1, 1), bij.reshape(-1, 1))
    print(loss)
    encorder_optimizer.zero_grad()
    decorder_optimizer.zero_grad()
    loss.backward()
    encorder_optimizer.step() 
    decorder_optimizer.step() 

In [481]:
def get_training_data(cur_n = 5000):
    g = nx.powerlaw_cluster_graph(n=cur_n, m=4, p=0.05)
    bc_value = nx.betweenness_centrality(g)
    dergees = {node:[val, 1, 1] for (node, val) in g.degree()}
    nx.set_node_attributes(g, dergees, "x")
    nx.set_node_attributes(g, bc_value, "y")
    
    return g
